In [2]:
### JALANKAN CELL INI TERLEBIH DAHULU ###
!pip install pandasql

In [3]:
import pandas as pd
import pandasql

from pandasql import sqldf
sql_run = lambda q: sqldf(q, globals())

In [4]:
worldcity = pd.read_csv('worldcities.csv', sep = ';', encoding = 'latin-1')
uscity = pd.read_csv('uscities.csv', sep = ';', encoding = 'latin-1')
uscity.columns = ['city', 'state_id', 'state_name', 'county_fips', 'county_name',
       'lat', 'lng', 'population', 'density', 'source', 'military',
       'incorporated', 'timezone', 'ranking', 'zips', 'id']

### Tutorial Bagaimana Cara Melakukan Query Menggunakan Pandasql

Contoh: Bagaimana melakukan query untuk mendapatkan kota-kota dari worldcity yang merupakan:
- berada di antara 33 - 40 'latitude'
- berada di antara 100 - 140 'longitude'
- merupakan kota 'primary' pada kolom 'capital'
- memiliki populasi lebih dari 20 juta

In [4]:
worldcity

,city,lat,lng,country,iso2,iso3,capital,population,id
0,Tokyo,35.6897,139.6922,Japan,JP,JPN,primary,37977000.0,1392685764
1,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,primary,34540000.0,1360771077
2,Delhi,28.6600,77.2300,India,IN,IND,admin,29617000.0,1356872604
3,Mumbai,18.9667,72.8333,India,IN,IND,admin,23355000.0,1356226629
4,Manila,14.6000,120.9833,Philippines,PH,PHL,primary,23088000.0,1608618140
...,...,...,...,...,...,...,...,...,...
40996,Tukchi,57.3670,139.5000,Russia,RU,RUS,NaN,10.0,1643472801
40997,Numto,63.6667,71.3333,Russia,RU,RUS,NaN,10.0,1643985006
40998,Nord,81.7166,-17.8000,Greenland,GL,GRL,NaN,10.0,1304217709
40999,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,NaN,10.0,1304206491


In [7]:
query = """
SELECT DISTINCT city
FROM worldcity
WHERE lat BETWEEN 33 AND 40
AND lng BETWEEN 100 AND 140
AND capital = 'primary'
AND population > 20000000
"""

sql_run(query)

,city
0,Tokyo
1,Seoul


### Question 1 Point : 40

Temukan kota-kota dari worldcity yang memenuhi semua kondisi ini:
- populasi kota tersebut melebih populasi dari kota terbanyak penduduknya di negara Filipina. (Maksudnya: misalnya kota X adalah kota di Filipina yang paling banyak penduduknya. Kita mau daftar kota-kota yang jumlah penduduknya lebih besar dari kota X tersebut), DAN
- ada kota di Amerika Serikat (uscity) dengan nama kota yang sama dengan kota-kota pilihan

Keluarkan hanya nama kota-kota yang memenuhi 2 syarat di atas saja, tidak usah kolom-kolom lain.

##### NOTE: Hilangkan duplicate rows.

Lakukan hanya 1x query saja, jangan melakukan lebih dari 1 query

In [19]:
query = """
SELECT DISTINCT wc.city
FROM worldcity wc
INNER JOIN (
    SELECT MAX(population) AS max_population
    FROM worldcity
    WHERE country = 'Philippines'
) AS philippine_max
ON wc.population > philippine_max.max_population
WHERE wc.city IN (
    SELECT DISTINCT wc2.city
    FROM worldcity wc2
    INNER JOIN uscity us ON wc2.city = us.city
)
"""

sql_run(query)

,city
0,Delhi


### Question 2 Point : 60

Temukan kota-kota di Amerika Serikat (uscity) yang memenuhi semua kondisi berikut:
- memiliki nama 'city' yang tidak sama dengan 'state_name', dan juga
- memiliki populasi di atas rata-rata populasi kota-kota di 'county_name' Miami-Dade, dan juga
- terdiri dari dua atau lebih kata (contoh: Los Angeles, New York)

Output yang diharapkan hanya kolom 'city' saja, tidak usah kolom yang lain

Lakukan hanya 1x query saja, jangan melakukan lebih dari 1 query

In [22]:
uscity.sample(3)

,city,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
21101,Gardner,FL,Florida,12049,Hardee,27.3537,-81.7895,340,17,polygon,False,False,America/New_York,3,33890,1840025167
28280,Ko Vaya,AZ,Arizona,4019,Pima,32.0815,-111.8959,8,2,polygon,False,False,America/Phoenix,3,85634,1840023072
28045,Roy Lake,MN,Minnesota,27029,Clearwater,47.3176,-95.5428,23,19,polygon,False,False,America/Chicago,3,56557,1840023822


In [5]:
query = """
SELECT DISTINCT city
FROM uscity
WHERE city <> state_name
    AND population > (
        SELECT AVG(population)
        FROM uscity
        WHERE county_name = 'Miami-Dade'
    )
    AND city LIKE '% %'
"""

sql_run(query)

,city
0,Los Angeles
1,San Francisco
2,San Diego
3,Las Vegas
4,San Antonio
...,...
84,South Lyon
85,Lehigh Acres
86,East Los Angeles
87,St. George
